# 12-ssda-xml-parser

> Parsing xml files to get a dataframe of this, that, and the other.

In this notebook, we create a parser that reads an xml file.  Our xml file contains a large amount of metadata about each of the different entries which we want to extract the entities from.  Each of the entries is within a book (with source identifier), and each book has a number of different folios (i.e., pages - a folio for the front and a separate one for the back), also with unique identifiers.  Since these are stored in XML format, we parse out the different components so we can create a dataframe of each of the entries, where each row is an entry with its respective metadata.

In [ ]:
# default_exp xml_parser

In [ ]:
#export
import pandas as pd

Below is an example of an xml file we'd have.  The header metadata and two entries are shown below:

In [ ]:
#no_test

four_xml = open("four.xml", "r", encoding='utf-8')
print(four_xml.read(2100))

<?xml version="1.0" encoding="utf-16"?>
<!--Libro 4 de Matrimonios de Pardos y Morenos, 1812-1839- ID: 1710-->
<ssda xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <volume>
    <volumeTitle>Libro 4 de Matrimonios de Pardos y Morenos, 1812-1839</volumeTitle>
    <volumeIdentifier>1710</volumeIdentifier>
    <volumeGuid>1a13417e-565e-4940-86a7-4d75abcea651</volumeGuid>
    <item>
      <itemTitle>Fol_1r</itemTitle>
      <itemIdentifier>1731</itemIdentifier>
      <itemGuid>e14ce5c5-2cc5-4384-ad42-fc5dc9177e0f</itemGuid>
      <text type="NormalizedText">
        <entry>
Libro 4.o en que se acientan las
partidas de matrimonios
de pardos y morenos de esta Iglesia
del Santo Angel Custodio de la ciudad
de la Havana, que comiensa en 18 de
Julio de 1812 á 1839.
Siendo #al Beneficiado el Licenciado don Andres
Cascales
</entry>
      </text>
    </item>
    <item>
      <itemTitle>Fol_2r</itemTitle>
      <itemIdentifier>1711</itemIdentifie

## Analysis and coding
The following code extracts the relevant parts of the metadata and constructs a dataframe which organizes the results by row.

In [ ]:
#export
def parse_xml(file_name):
    master_xml = open(file_name,"r",encoding='utf-8')
    vol_titls = []
    vol_ids = []
    entry_txts = []
    entry_ids = []
    fol_ids = []
    curr_vol_titl = ""
    curr_vol_id= ""
    curr_fol_id = ""
    curr_entry = ""

    in_entry = False

    for line in master_xml:    
        if (line.find('<') != -1) and (line.find('<', line.find('<') + 1) != -1):        
            line_content = line[line.find('>') + 1:line.find('<', line.find('<') + 1)]        
        elif line[len(line) - 2] == '-':
            line_content = line[:len(line) - 2]
        else:
            line_content = line[:len(line) - 1] + ' '
        
        if "<volumeTitle>" in line:
            curr_vol_titl = line_content
            #set current volume title
        elif "<volumeIdentifier>" in line:
            curr_vol_id = line_content
        #set current volume identifier
        elif "<itemIdentifier>" in line:
            entry_id = 0
            curr_fol_id = line_content
        #set current folio id
        elif "<entry>" in line:
            entry_id += 1
            in_entry = True
            curr_entry = ""
        #toggle in entry flag
        elif in_entry and (not "</entry>" in line):
            curr_entry += line_content
        #add line to current entry
        elif in_entry and ("</entry>" in line):
            in_entry = False 
        #toggle entry flag, append all current variables to lists
            vol_titls.append(curr_vol_titl)
            vol_ids.append(curr_vol_id)
            fol_ids.append(curr_fol_id)
            entry_txts.append(curr_entry)
            entry_ids.append(curr_fol_id + '-' + str(entry_id))
           
    columns = {'vol_titl':vol_titls, 'vol_id':vol_ids, 'fol_id':fol_ids, 'text':entry_txts, 'entry_no':entry_ids}
    
    df = pd.DataFrame(columns)
    master_xml.close()
    return df        

The following code shows the first 10 rows of the processed xml document:

In [ ]:
#no_test

parse_xml("four.xml").head(10)

,vol_titl,vol_id,fol_id,text,entry_no
0,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1731,Libro 4.o en que se acientan las partidas de m...,1731-1
1,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1711,Número 1 Pablo Ayende Maria Josefa Gomés En la...,1711-1
2,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1711,"N 2 Josef Rafael, Congo Juana Leandra Henrique...",1711-2
3,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1714,N 3 Josef Trinidad Mandi#garcía; y Maria de la...,1714-1
4,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1714,N 4 #mas Mariano #zales Mandinga Maria Luisa C...,1714-2
5,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1715,2. N 5 Placido Josef Acosta Maria Ynocen# Per...,1715-1
6,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1715,"N 6 Juan Nepomucemo Carabali, Maria Merced. Co...",1715-2
7,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1716,"N 7 Josef Placido Gonzales, y Marga rita Herna...",1716-1
8,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1716,"N 8 Marcelino Joaquin Bricamo, y Tomasa Crioll...",1716-2
9,"Libro 4 de Matrimonios de Pardos y Morenos, 18...",1710,1717,3. N 9 Josef Jesus Baesa Maria Antonia Lieche...,1717-1


In [ ]:
#export

def parse_xml_v2(path_to_xml):
    master_xml = open(path_to_xml,"r",encoding='utf-8')
    vol_titls = []
    vol_ids = []
    entry_txts = []
    entry_ids = []
    fol_ids = []
    curr_vol_titl = ""
    curr_vol_id= ""
    curr_fol_id = ""
    curr_entry = ""

    in_entry = False

    for line in master_xml:        
        if "<volume" in line:
            title_start = line.find('\"', line.find("title=")) + 1
            title_end = line.find('\"', title_start)
            curr_vol_titl = line[title_start:title_end]
            id_start = line.find('\"', line.find("id=")) + 1
            id_end = line.find('\"', id_start)
            curr_vol_id = line[id_start:id_end]       
        elif "<image" in line:
            entry_id = 0
            id_start = line.find('\"', line.find("id=")) + 1
            id_end = line.find('\"', id_start)
            curr_fol_id = line[id_start:id_end]                   
        elif "<entry" in line:
            entry_id += 1
            in_entry = True
            curr_entry = ""            
        elif in_entry and (not "</entry>" in line):
            if (len(curr_entry) > 0) and (curr_entry[-1] == '-'):
                curr_entry = curr_entry[:-1]
                curr_entry += line
            elif len(curr_entry) > 0:
                curr_entry += ' ' + line
            else:
                curr_entry += line
            curr_entry = curr_entry[:-1]
        elif in_entry and ("</entry>" in line):
            in_entry = False            
            vol_titls.append(curr_vol_titl)
            vol_ids.append(curr_vol_id)
            fol_ids.append(curr_fol_id)
            entry_txts.append(curr_entry)
            entry_ids.append(curr_fol_id + '-' + str(entry_id))
           
    columns = {'vol_titl':vol_titls, 'vol_id':vol_ids, 'fol_id':fol_ids, 'text':entry_txts, 'entry_no':entry_ids}
    
    df = pd.DataFrame(columns)
    master_xml.close()
    return df        

In [ ]:
#no_test

test_df = parse_xml_v2("transcriptions\\239746.xml")
test_df.head()

,vol_titl,vol_id,fol_id,text,entry_no
0,Baptisms - 1793-1807,239746,1013,"1. María Dolores Sanchez Lunes, día veinte y ...",1013-1
1,Baptisms - 1793-1807,239746,1013,"2. Antonio Guillo Miercoles, día veinte de No...",1013-2
2,Baptisms - 1793-1807,239746,1014,"3. María Juana Francisca Fish Domingo, día ve...",1014-1
3,Baptisms - 1793-1807,239746,1014,"4. Maria Teresa Camel Domingo, día veinte y q...",1014-2
4,Baptisms - 1793-1807,239746,1015,"Maria Josefa Andrea de la Puente Miércoles, d...",1015-1


In [ ]:
#export

def retrieve_volume_metadata(path_to_xml):
    xml = open(path_to_xml,"r",encoding='utf-8')
    volume_metadata = {}
    metadata_fields = ["type", "country", "state", "city", "institution", "id", "title"]    
    
    for line in xml:        
        if "<volume" in line:            
            for field in metadata_fields:                
                volume_metadata[field] = line[line.find('=', line.find(field)) + 2:line.find('\"', line.find('=', line.find(field)) + 2)]            
            
    return volume_metadata    

In [ ]:
#no_test

test = retrieve_volume_metadata("transcriptions\\15834.xml")
print(test)

{'type': 'baptism', 'country': 'Cuba', 'state': 'Matanzas', 'city': 'Matanzas', 'institution': 'Catedral de San Carlos Borromeo', 'id': '15834', 'title': 'Libro 1 de Bautismos de Pardos y Morenos, 1719 - 1752, Parroquia de San Carlos de Matanzas'}


In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 71-relationship-builder.ipynb.
